In [1]:
import pandas as pd
import pyodbc
import numpy as np
import pyreadr
import subprocess

In [9]:
path = r'T:\Tableau\tableauJourneyToWork\Datasources'

In [7]:
#https://appsilon.com/use-r-and-python-together/
subprocess.call([r"C:\Program Files\R\R-4.2.2\bin\Rscript", "--vanilla", "ModeShare.r"])

0

In [10]:
res = pyreadr.read_r(path+'\\mode_share_data.RData')

In [11]:
type(res)

collections.OrderedDict

In [12]:
print(res.keys())

odict_keys(['mode.share', 'mode.by.vehicles', 'mode.by.poverty', 'time.leaving', 'travel.time'])


In [13]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')

In [14]:
mode_share = pd.read_sql_query('select * from dbo.mode_share', conn)

In [16]:
mode_share.tail(3)

,year,geo,mode,estimate,moe_est,share,moe_share,share_pct,moe_share_pct
582,2021,Salem Urbanized Area,Walked,3067,436,0.025958,0.003653,2.595809,0.365340
583,2021,Salem Urbanized Area,Other means,782,255,0.006619,0.002154,0.661859,0.215417
584,2021,Salem Urbanized Area,Worked at home,11010,888,0.093185,0.007281,9.318505,0.728061


In [17]:
df = res['mode.share']

In [19]:
df.columns = mode_share.columns

In [22]:
fields = 'time_leaving, formatted_time'

In [25]:
field = 'mode'

In [24]:
fields.split(', ')

['time_leaving', 'formatted_time']

In [27]:
len(field.split(', '))

1

In [56]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [77]:
def update_data(ndf = df, tablenm = 'mode_share_test', fieldnm = 'mode', update = False):
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
    dat = pd.read_sql_query('select * from dbo.{}'.format(tablenm), conn)
    ndf.columns = dat.columns
    
    field_items = fieldnm.split(', ')
    
    if update:
        if len(field_items) == 2:
            for row in df.itertuples():
                cursor.execute('''
                INSERT INTO {0}(year, geo, {1}, estimate, moe_est, share, moe_share, share_pct, moe_share_pct) 
                VALUES (?,?,?,?,?,?,?,?,?)'''.format(tablenm, fieldnm),
                row.year,
                row.geo,
                eval(f'row.{field_items[0]}'),
            eval(f'row.{field_items[0]}'),
                row.estimate,
                row.moe_est,
                row.share,
                row.moe_share,
                row.share_pct,
                row.moe_share_pct)
        else:
            for row in df.itertuples():
                cursor.execute('''
                INSERT INTO {0}(year, geo, {1}, estimate, moe_est, share, moe_share, share_pct, moe_share_pct) 
                VALUES (?,?,?,?,?,?,?,?,?)'''.format(tablenm, fieldnm),
                row.year,
                row.geo,
                eval(f'row.{field_items[0]}'),
                row.estimate,
                row.moe_est,
                row.share,
                row.moe_share,
                row.share_pct,
                row.moe_share_pct)            
        conn.commit()
        print("data was updated")
            
    else:
        print("data was not updated")

In [52]:
def update_data(ndf = df, tablenm = 'mode_share_test', fieldnm = 'mode', update = False):
    conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=rliddb.int.lcog.org,5433;'
                      'Database=GIS_CLMPO;'
                      'Trusted_Connection=yes;')
    dat = pd.read_sql_query('select * from dbo.{}'.format(tablenm), conn)
    ndf.columns = dat.columns
    
    field_items = fieldnm.split(', ')
    
    if update:
        if len(field_items) == 2:
            for row in df.itertuples():
                eval(f'''cursor.execute('
                INSERT INTO {tablenm}(year, geo, {fieldnm}, estimate, moe_est, share, moe_share, share_pct, moe_share_pct) 
                VALUES (?,?,?,?,?,?,?,?,?)',
                       row.year,
                       row.geo,
                       row.{field_items[0]},
                       row.{field_items[1]},
                       row.estimate,
                       row.moe_est,
                       row.share,
                       row.moe_share,
                       row.share_pct,
                       row.moe_share_pct
                      )''')
        else:
            for row in df.itertuples():
                eval(f'''cursor.execute('
                INSERT INTO {tablenm}(year, geo, {fieldnm}, estimate, moe_est, share, moe_share, share_pct, moe_share_pct) 
                VALUES (?,?,?,?,?,?,?,?,?)',
                       row.year,
                       row.geo,
                       row.{field_items[0]},
                       row.estimate,
                       row.moe_est,
                       row.share,
                       row.moe_share,
                       row.share_pct,
                       row.moe_share_pct
                      )''')            
        conn.commit()
        print("data was updated")
            
    else:
        print("data was not updated")

In [78]:
update_data(update = True)

data was updated
